In [ ]:
# will create NN to get weight we can then copy other to the SNN small demo

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# --- Load and prepare data ---
iris = load_iris()
X = iris['data']
y = iris['target']

# Normalize input to [0, 1]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# One-hot encode output
y_cat = to_categorical(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# --- Custom activation function ---
def directional_push_activation(w=2.0):
    def activation(x):
        # Clip to avoid divide-by-zero
        x = tf.clip_by_value(x, 1e-6, 1.0 - 1e-6)
        return 1 / (1 + tf.pow((1 - x) / x, tf.exp(w / 3)))
    return activation

# Choose your activation weight
activation_weight = 2.0

# --- Build model ---
inputs = Input(shape=(4,))
x = Dense(10, activation=directional_push_activation(activation_weight))(inputs)
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- Train model ---
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# --- Evaluate ---
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

# --- Copy weights ---
weights = model.get_weights()

# If you want to use them elsewhere:
# model.set_weights(weights_copy)

# Optional: print layer weights
for i, w in enumerate(weights):
    print(f"Layer {i} weights shape: {w.shape}")


Test Accuracy: 0.9667
Layer 0 weights shape: (4, 10)
Layer 1 weights shape: (10,)
Layer 2 weights shape: (10, 3)
Layer 3 weights shape: (3,)


In [ ]:

my_input = np.array([[5.1, 3.5, 1.4, 0.2]])  # sepal length, sepal width, petal length, petal width

# need to standardize the inputs to between 0 and 1.0 ^ 


# Scale it using the same scaler
my_input_scaled = scaler.transform(my_input)

# Predict
prediction = model.predict(my_input_scaled)
predicted_class = np.argmax(prediction)

print(f"Predicted probabilities: {prediction}")
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted probabilities: [[0.6536397  0.25813422 0.08822606]]
Predicted class: 0


In [2]:
print(weights)

[array([[-0.30175677,  0.32245865, -0.4986203 ,  0.51874804,  0.7390744 ,
         0.84078616, -0.5497139 ,  0.12482596,  0.3937063 ,  0.18559742],
       [-0.51538515, -0.4123389 , -0.13548255, -0.66131175, -0.20388526,
         0.05857472, -0.47443748,  0.8218865 , -0.50879365, -0.5484046 ],
       [-0.6199794 ,  0.01200231,  0.5797131 ,  0.87162036,  0.76552474,
         0.8057634 , -0.061759  , -0.24822311,  0.53538615, -0.02949882],
       [-0.6475095 , -0.24158555, -0.47470486,  0.5815514 , -0.26259357,
         0.1167264 , -0.15303928,  0.05249047,  0.53752863, -0.61277384]],
      dtype=float32), array([ 0.        , -0.03341823,  0.        ,  0.22211385, -0.09098917,
       -0.12913936,  0.        ,  0.25118777, -0.19716568,  0.        ],
      dtype=float32), array([[ 0.42264903,  0.08506948, -0.01422855],
       [ 0.4458115 ,  0.10150149,  0.31266794],
       [ 0.6353699 ,  0.43023813,  0.28295106],
       [-0.9120095 ,  1.0877126 ,  0.5112515 ],
       [-0.7805683 , -0.11816